In [5]:
import pandas as pd
import numpy as np

df = pd.read_parquet('../data/full_imputed_n_index.parquet')
# 1. Identify all INDEX_ columns except the target
index_cols = [
    col for col in df.columns 
    if col.startswith('INDEX_') and col != 'INDEX_v2x_corr'
]

# 2. Drop those columns (this keeps INDEX_v2x_corr)
df = df.drop(columns=index_cols)

In [6]:
df.head()

,year,v2smorgavgact,project,v2elturnhog,v2ellodiseff,v2lgqugen,v2cauni,v2casurv,v2caacadfree,v2ellovttm,v3ststatag,v3ststybcov,v3stcensus,v2regopploc,v3pechilabl,codingstart,v3lgotovst,v2elpeace,v2ddsignci,v2ddlexci,v2ddadmci,v2ellovtsm,v2ddsigdrf,v2elloseat,v3lgdomchm,v2smorgelitact,v2ddadmrf,codingend_contemp,v2mefemjrn,v2ddpartrf,codingend,country_id,v2exfemhog,v2cltort,v2clkill,v2cltrnslw,v2clrspct,v2fsuffrage,v2ddthrepl,v2clfmove,...,v2canuni,v2exdeathog,v2regimpoppgroup,v2regoppgroupssize,v2ellobadis,v2regproreg,v2regantireg,v2regpower,v2regsuploc,v2edcentcurrlm,v2edcenttxbooks,v2edpoledprim,v2edpoledsec,v2edpoledrights,v2edpatriot,v2edideol,v2edplural,v2edcritical,v2edteautonomy,v2edmath,v2edtequal,v2temonitor,v2edteunion,v2edteunionindp,v2edtehire,v2edtefire,v2edscpatriot,v2edscpatriotcb,v2edscextracurr,v2medstateprint,v2medstatebroad,v2medpolstate,v2medpolnonstate,v2medpatriot,v2medentrain,v2ellobaseat,v2elloreseat,v2edideolch_rec,INDEX_v2x_corr,country_name
0,1950.0,NaN,0.0,NaN,NaN,0.0,1.0,-0.005,NaN,NaN,1.0,1.0,1.0,4.0,NaN,1789.0,NaN,-1.063,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024.0,10.0,NaN,2024.0,3.0,NaN,-0.512,-0.369,-0.155,-0.77,0.0,0.0,2.066,...,37.0,NaN,2.0,-1.737,NaN,2.0,10.0,2.0,1.0,0.516,1.518,1.685,1.432,0.71,0.541,-0.197,0.068,-0.716,0.257,1.205,-1.128,-0.733,1.168,-0.527,-0.07,-0.579,0.992,1.568,-0.365,-1.107,-1.673,-1.145,0.509,1.066,-0.071,NaN,NaN,0.34,0.819,Mexico
1,1951.0,NaN,0.0,NaN,NaN,0.0,1.0,-0.005,NaN,NaN,1.0,1.0,0.0,4.0,NaN,1789.0,NaN,-1.063,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024.0,10.0,NaN,2024.0,3.0,NaN,-0.512,-0.369,-0.155,-0.77,0.0,0.0,2.066,...,39.0,NaN,2.0,-1.737,NaN,2.0,10.0,2.0,1.0,0.516,1.518,1.685,1.432,0.71,0.541,-0.197,0.068,-0.716,0.257,1.205,-1.128,-0.733,1.168,-0.527,-0.07,-0.579,0.992,1.568,-0.365,-1.107,-1.673,-1.145,0.509,1.066,-0.071,NaN,NaN,0.34,0.819,Mexico
2,1952.0,NaN,0.0,1.0,NaN,0.0,1.0,-0.005,NaN,NaN,1.0,1.0,0.0,4.0,NaN,1789.0,NaN,-1.028,NaN,0.0,NaN,8.3,NaN,161.0,NaN,NaN,NaN,2024.0,10.0,NaN,2024.0,3.0,NaN,-0.512,-0.369,-0.155,-0.77,0.0,0.0,2.066,...,39.0,NaN,2.0,-1.737,NaN,2.0,10.0,2.0,1.0,0.516,1.518,1.685,1.432,0.71,0.541,-0.197,0.068,-0.716,0.257,1.205,-1.128,-0.733,1.168,-0.527,-0.07,-0.579,0.992,1.568,-0.365,-1.107,-1.673,-1.145,0.509,1.066,-0.071,NaN,NaN,0.34,0.819,Mexico
3,1953.0,NaN,0.0,NaN,NaN,0.0,1.0,-0.005,NaN,NaN,1.0,1.0,0.0,4.0,NaN,1789.0,NaN,-1.028,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024.0,10.0,NaN,2024.0,3.0,NaN,-0.512,-0.159,-0.155,-0.77,0.0,0.0,2.066,...,39.0,NaN,2.0,-1.737,NaN,2.0,10.0,2.0,1.0,0.516,1.518,1.685,1.432,0.71,0.541,-0.197,0.068,-0.716,0.257,1.205,-1.128,-0.733,1.168,-0.527,-0.07,-0.579,0.992,1.568,-0.365,-1.107,-1.673,-1.145,0.509,1.066,-0.071,NaN,NaN,0.34,0.819,Mexico
4,1954.0,NaN,0.0,NaN,NaN,0.0,1.0,-0.005,NaN,NaN,1.0,1.0,0.0,4.0,NaN,1789.0,NaN,-1.028,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024.0,10.0,NaN,2024.0,3.0,NaN,-0.512,-0.159,-0.155,-0.77,0.0,0.0,2.066,...,42.0,NaN,2.0,-1.737,NaN,2.0,10.0,2.0,1.0,0.516,1.518,1.685,1.432,0.71,0.541,-0.197,0.068,-0.716,0.257,1.205,-1.128,-0.733,1.168,-0.527,-0.07,-0.579,0.992,1.568,-0.365,-1.107,-1.673,-1.145,0.509,1.066,-0.071,NaN,NaN,0.34,0.819,Mexico


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import RidgeCV, LassoCV
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# --- 1. Prepare data, drop missing target ---
df_model = df.copy().dropna(subset=['INDEX_v2x_corr'])
y = df_model['INDEX_v2x_corr']
X = df_model.drop(columns=['INDEX_v2x_corr'])

# --- 2. Train/test split (hold-out same split for all models) ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# --- 3. Ridge & Lasso pipelines (automatic imputation + one-hot) ---
# Identify numeric vs. categorical
numeric_feats   = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_feats = ['country_name']

num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_transformer,   numeric_feats),
    ('cat', cat_transformer, categorical_feats)
])

# RidgeCV
ridge = RidgeCV(alphas=np.logspace(-3, 3, 100), cv=5)
pipe_ridge = Pipeline([
    ('preproc', preprocessor),
    ('ridge',   ridge)
])
pipe_ridge.fit(X_train, y_train)
pred_ridge = pipe_ridge.predict(X_test)

# LassoCV
lasso = LassoCV(cv=5, max_iter=10000)
pipe_lasso = Pipeline([
    ('preproc', preprocessor),
    ('lasso',   lasso)
])
pipe_lasso.fit(X_train, y_train)
pred_lasso = pipe_lasso.predict(X_test)

# --- 4. OLS via statsmodels (manual preproc) ---
# 4a. Select only numeric cols with at least one non-missing
valid_nums = [c for c in numeric_feats if df_model[c].notna().any()]

# 4b. Subset, fill country_name, one-hot encode
X_ols = X[valid_nums + categorical_feats].copy()
X_ols['country_name'] = X_ols['country_name'].fillna('missing')
X_ols = pd.get_dummies(X_ols, columns=['country_name'], drop_first=True)

# 4c. Mean-impute numeric
imp = SimpleImputer(strategy='mean')
X_ols[valid_nums] = imp.fit_transform(X_ols[valid_nums])

# 4d. Drop any leftover non-numeric (shouldn’t be any, but safe)
X_ols = X_ols.select_dtypes(include=[np.number])

# 4e. Split and fit
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(
    X_ols, y, test_size=0.20, random_state=42
)
ols_model = sm.OLS(y_train_o, sm.add_constant(X_train_o)).fit()
pred_ols   = ols_model.predict(sm.add_constant(X_test_o))

# --- 5. Compute metrics & assemble results ---
results = pd.DataFrame({
    'Model':      ['OLS', 'Ridge', 'Lasso'],
    'Test MSE':   [
        mean_squared_error(y_test_o, pred_ols),
        mean_squared_error(y_test,   pred_ridge),
        mean_squared_error(y_test,   pred_lasso)
    ],
    'Best Alpha': [
        None,
        pipe_ridge.named_steps['ridge'].alpha_,
        pipe_lasso.named_steps['lasso'].alpha_
    ]
})

print(results)

# --- 6. Plot comparison ---
plt.figure(figsize=(6,4))
plt.bar(results['Model'], results['Test MSE'])
plt.title('Test Set MSE: OLS vs Ridge vs Lasso')
plt.ylabel('Mean Squared Error')
plt.xlabel('Model')
plt.tight_layout()
plt.show()


/Users/paulotto/Developer/hertieAssignments/mlPaper/.venv/lib/python3.11/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['v3pechilabl' 'v3ellovtlg' 'v3lglegllo' 'v3eltvriguc' 'v3elageuc'
 'v3stcitlaw' 'v3elsec' 'v3eldirelc' 'v3eldireuc' 'v3eldirepr'
 'v3elncbmaj' 'v3elupvtlg' 'v3elupvtsm' 'v3elupstsl' 'v3elupseat'
 'v3elupstsm' 'v3ttlvote' 'v3psagefirst' 'v3psagesecond' 'v3psagethird'
 'v3psagepm' 'v3elwomcab' 'v3elncbpr' 'v3elagepr' 'v3elvotsml'
 'v3elloelsy' 'v3elparlel' 'v3eltrnout' 'v3elloseat' 'v3ellostlg'
 'v3elvotlrg' 'v3ellovtsm' 'v3ellostsl' 'v3ellostsm' 'v3ellostss'
 'v3exagehos' 'v3eltvrig' 'v3expathhs' 'v3lgelecup' 'v3lginelup'
 'v3lgleglup' 'v3lgello' 'v3lginello' 'v3lgqumin' 'v3lgintblo' 'v3lgamend'
 'v3lgamnsty']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/paulotto/Developer/hertieAssignments/mlPaper/.venv/lib/python3.11/site-packages/sklearn/linear_model/_r